In [ ]:
import os
import json

repo_path = os.path.join(os.path.dirname(os.path.abspath('')), '..')
config_path = os.path.join(repo_path, 'config', 'settings.json')
with open(config_path, 'r', encoding='utf-8') as f:
    config = json.load(f)

output_path = config.get('output', {}).get('path')
data_source_path =  config.get('datasource', {}).get('path')

wechat_bill_path = os.path.join(repo_path, data_source_path, config.get('datasource', {}).get("wechatpay_filemame"))
alipay_bill_path = os.path.join(repo_path, data_source_path, config.get('datasource', {}).get("alipay_filename"))
merged_bill_path = os.path.join(repo_path, output_path, config.get('output', {}).get('merged_filename'))

### 序列化并整理不同来源的账单，合并为一个规范化的账单

In [ ]:
import sys
import os
import pandas as pd

sys.path.append(os.path.join(os.path.dirname(os.path.abspath('')), 'utils'))
from utils.alipay_formatter import alipay_format
from utils.wechat_formatter import wechat_format

wechat_bill_df = pd.DataFrame()
if wechat_bill_path != "" and os.path.exists(wechat_bill_path):
    wechat_bill_df = wechat_format(wechat_bill_path)
    print(f"已完成微信账单格式化: {wechat_bill_path}")

alipay_bill_df = pd.DataFrame()
if alipay_bill_path != "" and os.path.exists(alipay_bill_path):
    alipay_bill_df = alipay_format(alipay_bill_path)
    print(f"已完成支付宝账单格式化: {alipay_bill_path}")


In [ ]:
# 按时间顺序合并账单
merged_bill_df = pd.concat([alipay_bill_df, wechat_bill_df], sort=True).sort_values(by='交易时间', ascending=True)

# 整理列顺序
column_order = ['交易时间', '类型', '金额(元)', '收/支', '支付方式', '交易对方', '商品名称', '备注']
merged_bill_df = merged_bill_df.reindex(columns=column_order)

# 过滤掉不计入收支的记录
merged_bill_df = merged_bill_df[merged_bill_df['收/支'] != '/']

merged_bill_df.to_csv(merged_bill_path, encoding='utf-8', index=False)
